In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import itertools
import regex as re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from textblob import TextBlob
import spacy
from spacy import displacy
import emoji
from pymongo import MongoClient
from flask import Flask, request 

### Mongodb

In [2]:
conn = MongoClient("localhost:27017")

db = conn.get_database("ironhack")

films = db.get_collection("film_reviews")

### IMDB

In [117]:
def sentimentAnalysis(sentence):
    sia = SentimentIntensityAnalyzer()
    pol = sia.polarity_scores(sentence)
    polarity = pol["compound"]
    return polarity


def review_sentiment(review_list, df):
    
    lista = [i for i in review_list]
    
    sia = SentimentIntensityAnalyzer()
    
    sentiment = [sentimentAnalysis(i) for i in lista]

    df["sentiment"] = sentiment

#### Obtener críticas

In [118]:
url = 'https://www.imdb.com/title/tt6723592/reviews?ref_=tt_urv'

def extract_critics_imdb(url, title):

    response = requests.get(url)

    soup = BeautifulSoup(response.content)

    test = soup.find_all('a', class_ = "title")

    IMDB = []

    for i in test :
        
        IMDB.append(i.text.strip())
        
    imdb = pd.DataFrame(IMDB)

    imdb['title'] = pd.Series(['Tenet' for x in range(len(imdb.index))])

    imdb.columns = ["review", "title"] 

    lista = [i for i in imdb.review]
    
    sia = SentimentIntensityAnalyzer()
    
    sentiment = [sentimentAnalysis(i) for i in lista]

    imdb["sentiment"] = sentiment
    
    return imdb



In [119]:
tenet = extract_critics_imdb(url, "Tenet")

### FILMAFFINITY

#### Obtener críticas

In [159]:
def extract_critics_fa(url, title):
    
    '''
    This funciton allows us to create a dataframe of the reviews users have made of a certain movie or tv show with just putting
    the url of the site and the corresponding title
    
    No two websites are the same, so it is important to remember that this funciton will only work for the Filmaffinity site, 
    and only if we enter in the "Show - more reviews", which can be found at the end of the first reviews available. Once inside
    the site, we copy the url and insert it in the function. 
    
    It is also importan to point out that with this method we will only get a limited number of reviews. In order to get all of
    them we would have to use Selenium, which will allow us to interact with the clicks in the website
    '''

    response = requests.get(url)

    soup = BeautifulSoup(response.content)

    test = soup.find_all('a', title = "ver fuente")

    nueva = []

    for i in test:

        nueva.append(i.text.strip())

    FA = [re.sub(r'\w*\ó\w\:\s\★*\½?\s\(\w*\s\w\)', '', i) for i in nueva]
    
    fa = pd.DataFrame(FA)
    
    fa['title'] = pd.Series([title for x in range(len(fa.index))])

    fa.columns = ["review", "title"]

    return fa

In [209]:
url = 'https://www.filmaffinity.com/es/pro-reviews.php?movie-id=827414'

lor = extract_critics_fa(url, "Vengadores: Infinity War")

In [210]:
insert_object(lor.review, "Vengadores: Infinity War")

'Film reviews succesfully added'

In [322]:
imdb.to_csv(r'C:\Users\juanp\Ironhack\proyectos\api-sentiment-project\data\imdb.csv' , index = False)

### DATAFRAME

In [156]:

def insert_object(review,title):

    '''
    This function inserts all the information from the dataframe we created with the previous function as a Mongodb object.
    This object will have two field: title (name of the film or show) and reviews (array with all the reviews users have made)
    '''
    
    test = []

    test = [i for i in review]
    
    datos = {'title': title,
        'reviews': test}

    films.insert_one(datos)
    
    return "Film reviews succesfully added"

In [121]:
insert_object(tenet.review, tenet.sentiment, "Tenet")

'Film reviews succesfully added'

### API  

#### URL for getting movie reviews from database

In [258]:
url_reviews = "http://localhost:5000/reviews/"
title = 'Tenet'

In [259]:
criticas = requests.get(url_reviews + title).json()

#### URL for adding new movie to database

In [238]:
url = "http://localhost:5000/new_review"

In [252]:
list = ["buenas", "tardes"]

In [253]:
datos = {'title': "Hello", 
        'reviews': list}

In [254]:
requests.post(url, datos)

<Response [200]>

#### URL for updating review of existing movie in database

#### URL for getting the sentiment analysis of movie reviews

In [211]:
url = "http://localhost:5000/sentiment/"
title = "Vengadores: Infinity War"

In [220]:
requests.get(url + title).json()

['positive = 17.94871794871795%',
 'negative = 23.076923076923077%',
 'neutral = 58.97435897435898%']

### Sentiment Analysis

In [230]:
def sentimentAnalysis(sentence):
    sia = SentimentIntensityAnalyzer()
    pol = sia.polarity_scores(sentence)
    polarity = pol["compound"]
    return polarity


def prueba(review_list):
    
    negative = 0
    
    positive = 0
    
    neutral = 0
    
    lista = [i for i in review_list]
    
    sentiment = [sentimentAnalysis(i) for i in lista]
    
    for i in sentiment:
        
        if i < -0.35:
        
            negative += 1
            
        elif i > 0.35:
            
            positive += 1
            
        else:
            
            neutral += 1
    
    
    pos = positive/len(sentiment)*100
    
    neg = negative/len(sentiment)*100
    
    neu = neutral/len(sentiment)*100
    
    
    result = [f'{pos}%', f'{neg}%',f'{neu}%']
    
    return result

In [236]:
lista = []


for i in r:
    
    lista.append(sentimentAnalysis(i))